In [1]:
import pandas as pd
import numpy as np

In [8]:
ccfile="data/credit_card/cc_info.csv"
txnfile="data/credit_card/transactions.csv"

cc=pd.read_csv(ccfile)
txn=pd.read_csv(txnfile,parse_dates=True)

In [6]:
cc.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [9]:
txn.head()


,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


# Question1
Your boss wants to identify those users that in your dataset never went above the monthly credit card limit (calendar month). The goal of this is to automatically increase their limit. Can you send him the list of Ids? 

In [23]:
#get month value based on date
import datetime
month=[]
len_d=len(txn['date'])
#print(len_d)

for i in range(len_d):
    date=txn.loc[i,"date"]
    date_formatted=datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    month_i=str(date_formatted.year)+'-'+str(date_formatted.month)
    month.append(month_i)

AttributeError: 'list' object has no attribute 'head'

In [78]:
def getMonth(date_str):
    date=datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return(str(date.year)+'-'+str(date.month))

In [79]:
txn['month2']=txn['date'].apply(getMonth)

In [81]:
txn.head()

,credit_card,date,transaction_dollar_amount,Long,Lat,month,month2
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-9,2015-9
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10,2015-10
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10,2015-10
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10,2015-10
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10,2015-10


In [25]:
txn['month']=month

In [39]:
txn.head()
monthlytxn=txn.groupby(["month","credit_card"])["transaction_dollar_amount"].sum()

In [40]:
monthlytxn.head()

month    credit_card     
2015-10  1003715054175576     8954.21
         1013870087888817    12090.34
         1023820165155391    19259.98
         1073931538936472     1638.03
         1077622576192810      604.84
Name: transaction_dollar_amount, dtype: float64

In [46]:
monthlytxn=pd.DataFrame(monthlytxn)
monthlytxn.reset_index(inplace=True)


In [52]:
monthlytxn.head()


,month,credit_card,transaction_dollar_amount
0,2015-10,1003715054175576,8954.21
1,2015-10,1013870087888817,12090.34
2,2015-10,1023820165155391,19259.98
3,2015-10,1073931538936472,1638.03
4,2015-10,1077622576192810,604.84


In [53]:
cc.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [56]:
data=monthlytxn.merge(cc,how="left",on="credit_card")

In [62]:
total_list=np.unique(data['credit_card'])

In [68]:
overlimit_list=data.loc[data['transaction_dollar_amount']>data['credit_card_limit'],'credit_card'].values

In [75]:
result=set(total_list)-set(overlimit_list)
print(len(result))
print(result)

862
{6465194079012865, 3742680218482691, 6698437277931525, 3707548197703686, 5275410446848007, 6303154567927815, 7728317158402059, 2519792944246796, 1818161941088271, 4876614541242383, 1664685555838993, 6245684915925010, 6959222140366866, 6325419957422105, 4290726060535834, 2239759218122779, 7195601645654041, 9681744051023898, 3306399358832671, 3702831934212129, 4803218554363943, 8553746017036332, 5814955316793393, 6514175342145589, 8779597037686839, 1222730774437945, 7452262324088889, 1363526625562684, 7838554896273468, 3259748360353854, 6075288092815423, 1461675388467265, 9043793084665932, 6780209767452752, 1169271395749970, 8765140629962839, 6457362521948249, 3417477890619483, 4954103748307036, 6872461016514653, 9472892860209244, 1582804611799136, 7180893798762595, 7581688456986723, 9466839319443564, 7030584443306096, 8822567299944562, 4386321242706043, 2089905184376958, 5930630722779265, 2698111742263426, 5617658887579781, 9758203643408520, 6975663682594960, 7651566467203216, 12952

# question 2
On the other hand, she wants you to implement an algorithm that as soon as a user goes above her monthly limit, it triggers an alert so that the user can be notiﬁed about that.We assume here that at the beginning of the new month, user total money spent gets reset to zero (i.e. she pays the card fully at the end of each month).  Build a function that for each day, returns a list of users who went above their credit card monthly limit on that day. 

In [86]:
data2=txn.merge(cc,how="left",on="credit_card")

In [99]:
data2.head()
from datetime import timedelta  

datetime.datetime(2015, 9, 11, 0, 0)

In [129]:
def toDateTime(str):
    return(datetime.datetime.strptime(str, "%Y-%m-%d %H:%M:%S"))

In [128]:
#given month and date, calculate rolling total for each credit card and get the ones which exceed limit 
def calRollingToal(df,d_month,d_next):    
    focusdata=df.loc[(df["month"]==d_month)&(df["date"]<d_next),:]
    focusdata1=focusdata.groupby("credit_card").sum()
    focusdata2=focusdata1["transaction_dollar_amount"].reset_index()
    focusdata3=focusdata2.merge(cc,how="left",on="credit_card")
    result=focusdata3.loc[focusdata3["transaction_dollar_amount"]>focusdata3["credit_card_limit"],:]
    return(result["credit_card"].values)    

In [ ]:
data2["date"]=data2["date"].apply(toDateTime)

In [140]:
d = datetime.datetime(2015, 8, 20) 
d_next=d+ timedelta(days=1)
d_month=str(d.year)+'-'+str(d.month)
d_next
 

datetime.datetime(2015, 8, 21, 0, 0)

In [143]:
#assume input date is datetime format
def alertFunc(date):
    d=date
    d_next=d+ timedelta(days=1)
    d_month=str(d.year)+'-'+str(d.month)
    
    list1=calRollingToal(data2,d_month,d)
    list2=calRollingToal(data2,d_month,d_next)
    return(set(list2)-set(list1))

In [144]:
d1 = datetime.datetime(2015, 8, 20) 
alertFunc(d1)

{5723635641134781, 7299183791723634, 8972201384562696}

# Question 3
Finally, your boss is very concerned about frauds cause they are a huge cost for credit card companies. She wants you to implement an unsupervised algorithm that returns all transactions that seem unusual and are worth being investigated further.